In [1]:
import occRobotics
import numpy as np
from OCC.Core import gp

# Test makeTrsf

In [5]:
R = occRobotics.eul2R_zyx(0,0,np.pi/4) 
t = np.array([1,2,3])
gpT = occRobotics.makeTrsf(R,t)
gpT.Invert()

print(gpT.TranslationPart().Coord())
print(occRobotics.gpMat3toNumpy(gpT.GetRotation().GetMatrix()))

(0.9999999999999999, 2.0, 3.0)
[[ 0.70710678 -0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]]
